In [ ]:
from pyskl.models import build_model
from mmcv import Config
from pyskl.datasets.pipelines import Compose
import seaborn as sns
from mmcv.runner.checkpoint import load_checkpoint
from pyskl.datasets import build_dataloader, build_dataset
import torch
import matplotlib.pyplot as plt
import numpy as np


In [ ]:
with open('../tools/data/label_map/nturgbd_120.txt', 'r') as file:
    label_names = [line.strip() for line in file]

In [ ]:
import matplotlib.pyplot as plt
import time
import matplotlib.colors as mcolors
import mpl_toolkits.mplot3d.axes3d as p3

def Vis3DPose(keypoint, graph, label, index, angle=(30, 45), fig_size=(8, 8), with_grid=False):
    kp = keypoint
    colors = ('#3498db', '#000000', '#e74c3c')  # l, m, r

    graph = graph.max(0)
    flattened_array = graph.flatten()
    sorted_indices = np.argsort(flattened_array)[::-1]
    top_50_indices_flattened = sorted_indices[:20]
    values = flattened_array[top_50_indices_flattened]
    # Calculate min and max of the array
    min_A = np.min(values)
    max_A = np.max(values)
    # Scale the array to be between 0 and 1
    values = (values - min_A) / (max_A - min_A)
    colors_50 = plt.cm.Wistia(values)
    top_50_indices_2d = np.unravel_index(top_50_indices_flattened, graph.shape)

    plt.cm.Oranges([0,1,3,4])

    num_joint = 25
    kinematic_tree = [
        [1, 2, 21, 3, 4],
        [21, 9, 10, 11, 12, 25], [12, 24],
        [21, 5, 6, 7, 8, 23], [8, 22],
        [1, 17, 18, 19, 20],
        [1, 13, 14, 15, 16]
    ]
    kinematic_tree = [[x - 1 for x in lst] for lst in kinematic_tree]
    colors = ['black', 'blue', 'blue', 'red', 'red', 'darkblue', 'darkred']
    colors = ['blue', 'blue', 'blue', 'blue', 'blue', 'blue', 'blue']

    assert len(kp.shape) == 4 and kp.shape[3] == 3 and kp.shape[2] == num_joint
    x, y, z = kp[..., 0], kp[..., 1], kp[..., 2]
    min_x, max_x = min(x[x != 0]), max(x[x != 0])
    min_y, max_y = min(y[y != 0]), max(y[y != 0])
    min_z, max_z = min(z[z != 0]), max(z[z != 0])

    max_axis = max(max_x - min_x, max_y - min_y, max_z - min_z)
    mid_x, mid_y, mid_z = (min_x + max_x) / 2, (min_y + max_y) / 2, (min_z + max_z) / 2

    min_x, max_x = mid_x - max_axis / 2, mid_x + max_axis / 2
    min_y, max_y = mid_y - max_axis / 2, mid_y + max_axis / 2
    min_z, max_z = mid_z - max_axis / 2, mid_z + max_axis / 2

    fig = plt.figure(figsize=fig_size)
    ax = p3.Axes3D(fig)

    ax.set_xlim3d([min_x, max_x])
    ax.set_ylim3d([min_y, max_y])
    ax.set_zlim3d([min_z, max_z])
    ax.view_init(*angle)
    # fig.suptitle(item.get('frame_dir', 'demo'), fontsize=20)
    def update(t):
        # ax.lines = []
        ax.view_init(*angle)
        for i, (chain, color) in enumerate(zip(kinematic_tree, colors)):
            linewidth = 2.0
            for j in range(kp.shape[0]):
                ax.plot3D(kp[j, t, chain, 0], kp[j, t, chain, 1], kp[j, t, chain, 2], linewidth=linewidth, color=color)
        for chain, rgba_color in zip(top_50_indices_2d, colors_50):
            linewidth = 1.0
            hex_color = mcolors.to_hex(rgba_color)
            for j in range(kp.shape[0]):
                ax.plot3D(kp[j, t, chain, 0], kp[j, t, chain, 1], kp[j, t, chain, 2], linewidth=linewidth, color=hex_color)
        if not with_grid:
            plt.axis('off')
            ax.set_xticklabels([])
            ax.set_yticklabels([])
            ax.set_zticklabels([])
    update(50)
    plt.title(label_names[label])
    # plt.show()
    name = label_names[label].replace("/", " or ")
    plt.savefig(f'attn_vis/{name}_{index}.png')

In [ ]:
base = '../work_dirs/aagcn/aagcn_pyskl_ntu60_xsub_3dkp/j_0.01_random8_convnew'
config = f'{base}/j.py'
checkpoint_file = f'{base}/best_top1_acc_epoch_16.pth'
# best_top1_acc_epoch_16

In [ ]:
base2= '../work_dirs/aagcn/aagcn_pyskl_ntu60_xsub_3dkp/j_0.01_random8_convnew_gcl'
config2 = f'{base2}/j.py'
checkpoint_file2 = f'{base2}/epoch_16.pth'
# best_top1_acc_epoch_16

In [ ]:
cfg = Config.fromfile(config)
cfg.data.val['ann_file'] = '../data/nturgbd/ntu60_3danno.pkl'
cfg.model['backbone']['return_graph'] = True
model = build_model(cfg.model)
val_dataset = build_dataset(cfg.data.val)
load_checkpoint(model, checkpoint_file)


In [ ]:
# cfg2 = Config.fromfile(config2)
# cfg2.model['backbone']['return_graph'] = True
# model2 = build_model(cfg2.model)
# load_checkpoint(model2, checkpoint_file2)

In [ ]:
i = 10
keypoints = val_dataset[i]['keypoint']
label = val_dataset[i]['label']
print(label_names[label])
with torch.no_grad():
    x, graph = model.backbone(keypoints)
    graph = graph.numpy().reshape((-1, 25, 25))
    # x2, graph2 = model2.backbone(keypoints)
    # graph2 = graph2.numpy().reshape((-1, 25, 25))

In [ ]:
# layer = 8

# plt.figure(figsize=(16, 9))

# for i in range(8):
#     data = graph[i]
#     # Create a heatmap
#     plt.subplot(2, 4, i + 1)
#     plt.imshow(data, cmap='Blues', interpolation='nearest')
#     plt.title(f"Matrix {i}")

# # Show the plot
# plt.show()

In [ ]:
# for i in range(100):
#     keypoints = val_dataset[i]['keypoint']
#     label = val_dataset[i]['label']
#     print(label_names[label])
#     with torch.no_grad():
#         x, graph = model.backbone(keypoints)
#         graph = graph.numpy().reshape((-1, 25, 25))
#         # x2, graph2 = model2.backbone(keypoints)
#         # graph2 = graph2.numpy().reshape((-1, 25, 25))
#         Vis3DPose(keypoints.numpy()[0], graph, label, i, angle=(30, 45), fig_size=(8, 8), with_grid=False)

In [ ]:
Vis3DPose(keypoints.numpy()[0], graph, label, 0, angle=(30, 45), fig_size=(8, 8), with_grid=False)